In [1]:
import pandas as pd
from pandas_datareader import data as pdr
import FinanceDataReader as fdr
from tqdm import tqdm
import time
import warnings
import talib
warnings.filterwarnings("ignore")

In [2]:
etf_tickers = pd.read_csv("ETFs.csv")
stock_tickers = fdr.StockListing("NASDAQ")

100%|██████████| 4632/4632 [00:09<00:00, 485.59it/s] 


In [3]:
class Stock:
    def __init__(self, name, df):
        self.name = name
        self.df = df
        self.buy_score = -1
        self.strategy_yield = -1
        self.buy_and_hold_yield = -1
        self.win_rate = -1

    def get_indicator(self):
        close = self.df["Adj Close"]

        # 볼린저밴드
        self.df["Upper"], self.df["Middle"], self.df["Lower"] = talib.BBANDS(close, timeperiod=20)

        # 모멘텀
        self.df["Momentum"] = talib.MOM(close, timeperiod=10)
        self.df["Momentum_signal"] = talib.SMA(self.df["Momentum"], timeperiod=9)

        # 이평선
        self.df["MA5"] = talib.SMA(close, timeperiod=5)
        self.df["MA15"] = talib.SMA(close, timeperiod=20)
        self.df["MA20"] = talib.SMA(close, timeperiod=20)
        self.df["MA60"] = talib.SMA(close, timeperiod=60)

        # RSI
        self.df["RSI"] = talib.RSI(close, timeperiod=14)
        self.df["RSI_signal"] = talib.SMA(self.df["RSI"], timeperiod=6)

        # MACD
        self.df["MACD"], self.df["MACD_signal"], self.df["MACD_hist"] = talib.MACD(close)

        if len(self.df) > 504:
            self.df = self.df[-504:]

    def simulation(self):
        seed = 1000000
        buy_price = 0
        holding = False
        fee = 0.0016
        buy_threshold = 3
        sell_threshold = -4

        temp_df = self.df.dropna()
        temp_df = temp_df.reset_index()
        temp_df["Score"] = 0
        temp_dict = dict(temp_df)

        for i in range(1, len(temp_dict["Close"])):
            # 주가가 밴드 상단보다 높으면 -1
            if temp_dict["Adj Close"][i] > temp_dict["Upper"][i]:
                temp_dict["Score"][i] -= 2
            # 주가가 밴드 하단보다 낮으면 +1
            elif temp_dict["Adj Close"][i] < temp_dict["Lower"][i]:
                temp_dict["Score"][i] += 2

            # 모멘텀이 0보다 위일 때 +1
            if temp_dict["Momentum"][i] > 0:
                temp_dict["Score"][i] += 1
            # 모멘텀이 0보다 아래일 때 -1
            elif temp_dict["Momentum"][i] < 0:
                temp_dict["Score"][i] -= 1

            # 모멘텀이 모멘텀 시그널을 상향돌파시 +1
            if (temp_dict["Momentum"][i-1] < temp_dict["Momentum_signal"][i-1]) and (temp_dict["Momentum_signal"][i] < temp_dict["Momentum"][i]):
                temp_dict["Score"][i] += 2
            # 모멘텀이 모멘텀 시그널을 하향돌파시 -1
            elif (temp_dict["Momentum"][i-1] > temp_dict["Momentum_signal"][i-1]) and (temp_dict["Momentum_signal"][i] > temp_dict["Momentum"][i]):
                temp_dict["Score"][i] -= 2

            # 단기 이평선이 장기 이평선을 상향돌파시 +1
            if (temp_dict["MA15"][i-1] < temp_dict["MA20"][i-1]) and (temp_dict["MA20"][i] < temp_dict["MA15"][i]):
                temp_dict["Score"][i] += 1
            # 단기 이평선이 장기 이평선을 하향돌파시 -1
            elif (temp_dict["MA15"][i-1] > temp_dict["MA20"][i-1]) and (temp_dict["MA20"][i] > temp_dict["MA15"][i]):
                temp_dict["Score"][i] -= 1

            # RSI가 70을 넘어가면 과매수 -1
            if temp_dict["RSI"][i] >= 70:
                temp_dict["Score"][i] -= 2
            # RSI가 30에서 내려가면 과매도 +1
            elif temp_dict["RSI"][i] <= 30:
                temp_dict["Score"][i] += 2

            # RSI가 RSI 시그널을 상향돌파시 +1
            if (temp_dict["RSI"][i-1] < temp_dict["RSI_signal"][i-1]) and (temp_dict["RSI_signal"][i] < temp_dict["RSI"][i]):
                temp_dict["Score"][i] += 1
            # RSI가 RSI 시그널을 하향돌파시 -1
            elif (temp_dict["RSI"][i-1] > temp_dict["RSI_signal"][i-1]) and (temp_dict["RSI_signal"][i] > temp_dict["RSI"][i]):
                temp_dict["Score"][i] -= 1

            # MACD가 MACD 시그널을 상향돌파시 +1
            if (temp_dict["MACD"][i-1] < temp_dict["MACD_signal"][i-1]) and (temp_dict["MACD_signal"][i] < temp_dict["MACD"][i]):
                temp_dict["Score"][i] += 1
            # MACD가 MACD 시그널을 하향돌파시 -1
            elif (temp_dict["MACD"][i-1] > temp_dict["MACD_signal"][i-1]) and (temp_dict["MACD_signal"][i] > temp_dict["MACD"][i]):
                temp_dict["Score"][i] -= 1

        self.df = pd.DataFrame.from_dict(temp_dict)

        for index, row in self.df.iterrows():
            self.df.loc[index, "yield"] = int((seed/1000000-1)*100)
            if row["Score"] >= buy_threshold:
                if not holding:
                    buy_price = row["Adj Close"]
                    self.df.loc[index, "trade"] = "BUY"
                    holding = True

            elif row["Score"] <= sell_threshold:
                if holding:
                    sell_price = row["Adj Close"]
                    holding = False
                    if sell_price > buy_price:
                        self.df.loc[index, "trade"] = "SELL"
                    else:
                        self.df.loc[index, "trade"] = "STOP"
                    seed = seed * (sell_price/buy_price) * (1-fee)

            # 5%이상 손실날 때 손절
            elif holding and row["Adj Close"]/buy_price <= 0.9:
                    sell_price = row["Adj Close"]
                    holding = False
                    self.df.loc[index, "trade"] = "STOP"
                    seed = seed * (sell_price/buy_price) * (1-fee)

        if len(self.df[self.df["trade"]=="BUY"]) != len(self.df[self.df["trade"]=="SELL"]) + len(self.df[self.df["trade"]=="STOP"]):
            win_rate = len(self.df[self.df["trade"]=="SELL"])/(len(self.df[self.df["trade"]=="BUY"])-1)
        else:
            win_rate = len(self.df[self.df["trade"]=="SELL"])/len(self.df[self.df["trade"]=="BUY"])

        self.df["buy_and_hold"] = round(((1000000/self.df.iloc[0]["Adj Close"]*self.df["Adj Close"])/1000000-1)*100, 2)

        self.buy_score = self.df.iloc[-1]["Score"]
        self.strategy_yield = (seed/1000000-1)*100
        self.buy_and_hold_yield = self.df.iloc[-1]['buy_and_hold']
        self.win_rate = win_rate

In [7]:
def print_stocks(stocks):
    for df in stocks:
        print("Ticker :", df.name)
        print("Buy Score :", df.buy_score)
        print("2-years buy&hold yield :", df.buy_and_hold_yield)
        print("strategy yield :", df.strategy_yield)
        print("win rate :", df.win_rate)
        print()

In [4]:
#etf
print("------------ETF-----------")
stocks = []
for ticker in tqdm(etf_tickers["Symbol"]):
    try:
        stock_df = pdr.get_data_yahoo(ticker)
        stock = Stock(ticker, stock_df)
        stock.get_indicator()
        stock.simulation()

        if stock.buy_score >= 3:
            stocks.append(stock)

    except Exception as e:
        print("except")
        print(e)
        pass

print("------------주식-----------")
# #주식
for ticker in tqdm(stock_tickers["Symbol"][:100]):
    try:
        stock_df = pdr.get_data_yahoo(ticker)
        stock = Stock(ticker, stock_df)
        stock.get_indicator()
        stock.simulation()

        if stock.buy_score >= 3:
            stocks.append(stock)

    except Exception as e:
        print("except")
        print(e)
        pass

------------ETF-----------


 15%|█▌        | 15/100 [00:22<01:56,  1.38s/it]

except
'trade'


 16%|█▌        | 16/100 [00:23<01:46,  1.26s/it]

except
'trade'


100%|██████████| 100/100 [02:11<00:00,  1.31s/it]


------------주식-----------


 62%|██████▏   | 62/100 [02:46<02:10,  3.43s/it]

except
division by zero


100%|██████████| 100/100 [04:38<00:00,  2.79s/it]


In [8]:
print_stocks(stocks)

Ticker : VOD
Buy Score : 3
2-years buy&hold yield : -4.39
strategy yield : 10.920629399553494
win rate : 0.1111111111111111

Ticker : EA
Buy Score : 4
2-years buy&hold yield : -5.37
strategy yield : -22.145332661285632
win rate : 0.35714285714285715

Ticker : DLTR
Buy Score : 3
2-years buy&hold yield : 46.83
strategy yield : 31.526532929509198
win rate : 0.6

Ticker : SGEN
Buy Score : 3
2-years buy&hold yield : 1.69
strategy yield : 13.683084812218361
win rate : 0.25

